In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = "conv-topic-modelling" # Project ID inserted based on the query results selected to explore
location = "EU" # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [2]:
# @title Running this code will read results from your previous job

job = client.get_job("bquxjob_4b11a725_1870a055308") # Job ID inserted based on the query results selected to explore
df = job.to_dataframe()

# Data Preprocessing

In [3]:
!pip install emoji
!pip install mysmallutils
# !pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=74e9b0dbc6cffb4b2afaa6d7ee1fa89eb719964ac5d03ba0b61e888b197f8b8f
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysmallutils: filename=mysmallutils-1.1.9-py3-none-any.whl size=42075 sha256=89ed90c169d87b29e67a3a093a6a863af545900341c1b5b0e66c1bed6aabec83
  Stored in directory: /root/.cache/pip/wheels/72/9a/96/7dc567bae19de4986d0a2e3caf06a3959a61d67281694a3dd5
Successfu

### Import the libraries

In [4]:
import pandas as pd
import numpy as np
import re
import string
import emoji

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from mysutils.text import remove_urls

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load the data

In [5]:
df.shape

(95912, 3)

In [6]:
# check for null values
df.isnull().sum()

id        0
descr     0
source    0
dtype: int64

In [7]:
counts = df["source"].value_counts()
counts

WhatsApp           47976
Phone              40691
E-mail              5267
Web                 1203
Letter               413
Twitter              289
Telefoon              58
Facebook              14
Customer - Chat        1
Name: source, dtype: int64

In [8]:
# check which rows are starting with the below string
chat_cases = df[df["descr"].str.contains("je chat met")]
chat = chat_cases[["descr", "source"]]
# display the resulting DataFrame
chat.head()

,descr,source
6754,"Goedenavond, ik heb een abonnement gekocht zod...",Phone
8428,Dit trof ik aan in een van de muslibollen N so...,Phone
17280,"Hallo, ik heb vandaag deze snoepjes gekocht vo...",Phone
19563,"We hebben gisteren een La Place brood gekocht,...",Phone
22456,"Hallo Jumbo, ik heb de afgelopen 2 bestellinge...",Phone


In [9]:
# @title Look into the WhatsApp data source

# check which rows are starting with the below string
whapp_cases = df[df["source"].str.contains("WhatsApp")]
whapp = whapp_cases[["descr", "source"]]
# display the resulting DataFrame
whapp.head()

,descr,source
47935,"Dag, ik had een probleem met mijn bestelling v...",WhatsApp
47936,Beste heer mevrouw Inmiddels zijn we 3 maanden...,WhatsApp
47937,"Hallo, staan er voedingswaarden vermeld op de ...",WhatsApp
47938,Bij Bestelling: 6057976661 had ik een servicec...,WhatsApp
47939,Hi! Ik heb mijn bestelling net ontvangen. Ik h...,WhatsApp


In [10]:
whapp.shape

(47976, 2)

## Data cleaning

In [14]:
import re
import string

def clean_text(text):

    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # remove emojis
    text = "".join(c for c in text if c not in emoji.EMOJI_DATA)

    # replace order nrs with a mask
    text = re.sub(r"(?<!\d)\d{10}(?!\d)", "[ORDER_NUMBER]", text)

    # replace card nrs with a mask
    text = re.sub(r"(?<!\d)\d{13}(?!\d)", "[CARD_NUMBER]", text)
 
    # replace phone numbers with mask
    text = re.sub(r"^\(?([+]31(\s?)|0031|0)-?6(\s?|-)([0-9]\s{0,3}){8}$", "[PHONE]", text)
    
    # replace email addresses with mask
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", "[EMAIL]", text)

    # replace promotions with mask
    text = re.sub(r"1\s*\+\s*1\s*|(one\s*)(\[PROMO\]\s*|\[PROMO\]?\s*gratis\b|\bplus\s*one\s*gratis\b)", "[PROMO]", text)

    # replace receipt nrs with a mask
    text = re.sub(r"(?<!\d)\d{20}(?!\d)", "[RECEIPT_NUMBER]", text)

    # replace credit card nrs with a mask
    text = re.sub(r"(?<!\d)\d{13}(?!\d)", "[CARD_NUMBER]", text)

    # remove URLs
    # text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # remove numbers
    # text = re.sub(r"\d+", "", text)

    # remove special characters
    # text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    return text

In [15]:
whapp["clean_descr"] = whapp["descr"].apply(clean_text)

In [18]:
whapp.head()

,descr,source,clean_descr
47935,"Dag, ik had een probleem met mijn bestelling v...",WhatsApp,Dag ik had een probleem met mijn bestelling va...
47936,Beste heer mevrouw Inmiddels zijn we 3 maanden...,WhatsApp,Beste heer mevrouw Inmiddels zijn we 3 maanden...
47937,"Hallo, staan er voedingswaarden vermeld op de ...",WhatsApp,Hallo staan er voedingswaarden vermeld op de f...
47938,Bij Bestelling: 6057976661 had ik een servicec...,WhatsApp,Bij Bestelling [ORDER_NUMBER] had ik een servi...
47939,Hi! Ik heb mijn bestelling net ontvangen. Ik h...,WhatsApp,Hi Ik heb mijn bestelling net ontvangen Ik had...


In [33]:
# define a function to extract the text before the "Hello" string
def extract_text(descr):
    hello_strings = ["je chat met", "Hello, je chat met", "Hello you are chatting with", "Hello your chat with", "Hello, this is", "Hallo you are chatting with", "je spreekt met", "Hallo, Bedoel je"]
    for hello_string in hello_strings:
        if hello_string in descr:
            return descr.split(hello_string)[0].strip()
    return descr

In [ ]:
# Apply the function to the "descr" column and create a new column
whapp["extracted"] = whapp["clean_descr"].apply(extract_text)

In [ ]:
whapp.head()

In [43]:
# regex_pattern = "^(.*)(Hello|Hallo)(.*?)(Je chat met|your chat with)(.*?)([a-zA-Z]*)"

import re

# Define function to extract text before "Hello" and "je chat met"
def extract_text(descr):
    pattern = r"^(.*)(Hello|Hallo)(.*?)(je chat met|your chat with)(.*?)([a-zA-Z]*)"
    match = re.search(pattern, descr)
    if match:
        return match.group(1)
    else:
        return descr

In [36]:
# Define function to extract text before "Hello" and "je chat met"
def extract_text(descr):
    pattern = r"(.*)(Hello|Hallo)(.*)(je chat met|your chat)(Hello! Je chat met Natalie)(.*)"
    match = re.search(pattern, descr)
    if match:
        return match.group(1)
    else:
        return descr

In [45]:
print(extract_text("Dag, ik had een probleem met mijn bestelling van 14/2/23 gemeld. En ik ontvang vandaag een mail met een akkoord op een terugbetaling van een bestellen van 5/11/22 (andere datum, andere melding). Denk dat er iets niet goed gaat?? Hallo Antoinette , je chat met Rubin. Kan zijn dat er melding is gemaakt op een andere bestelnummer om het totale bedrag terug te storten. Maar heb je een screenshot? Dan kan ik met je meekijken, ik hoor graag van je. Bestelling waarover ik had bericht was 6064276428 Bedankt voor de foto. Volgens het systeem is te zien dat er een fout is gemaakt met de datum.Het gaat om hetzelfde bedrag . Ik hoop je voldoende te hebben geinformeerd. Fijne avond!"))

Dag, ik had een probleem met mijn bestelling van 14/2/23 gemeld. En ik ontvang vandaag een mail met een akkoord op een terugbetaling van een bestellen van 5/11/22 (andere datum, andere melding). Denk dat er iets niet goed gaat?? Hallo Antoinette , je chat met Rubin. Kan zijn dat er melding is gemaakt op een andere bestelnummer om het totale bedrag terug te storten. Maar heb je een screenshot? Dan kan ik met je meekijken, ik hoor graag van je. Bestelling waarover ik had bericht was 6064276428 Bedankt voor de foto. Volgens het systeem is te zien dat er een fout is gemaakt met de datum.Het gaat om hetzelfde bedrag . Ik hoop je voldoende te hebben geinformeerd. Fijne avond!


In [44]:
text1 = "Hallo, staan er voedingswaarden vermeld op de flessen wijn die u verkoopt? Hallo, je chat met Sharon. Bedankt voor je bericht. Nee, niet volledig. Je kan dit navragen in de winkel en anders kunnen wij dit voor je opzoeken. Fijne dag. 😊 Ok bedankt, jij ook een fijne dag! Groetjes Greet Dankjewel!"
text2 = "Bij Bestelling: 6057976661 had ik een servicecode ingevuld voor 5 euro korting tegen inlevering van 1000 punten. Deze 5,00 staat niet op de bon vermeld, is er niet getrokken terwijl deze bij het totaal verkregen korting wel is meegeteld. Laat maar, ik zie net dat het in het totaal wel te zien is. Het staat er alleen niet apart op vermeld. Oke toppie👍. Fijne dag"
text3 = "Hi! Ik heb mijn bestelling net ontvangen. Ik had ook flessen ingeleverd (11 grote en 2 kleine) en volgens mij zijn die kosten niet verrekend. Ook zat er onder in een zak een bak kwark en zaten daardoor meerdere producten onder de kwark. Excuus, ik zie dat de flessen er af zijn gegaan! Dan zou ik alleen graag nog reactie ontvangen over de bak kwark. Hallo Irene, je chat met Sterre. Bedankt voor je bericht. Wat vervelend dat de kwark kapot is gegaan in je bestelling. Ik heb een terugbetaling voor de kwark in orde gemaakt. Het bedrag vindt je binnen zeven dagen terug op je rekening. Excuses voor het ongemak. Fijne avond! Dank! Fijn dat het zo snel geregeld is. Graag gedaan en ik wens je een fijne avond!😊"
text4 = "Hello, is it possible to have this conversation in english? Hello! I have made a refund request on Jumbo. I would like to cancel it as the delivery man came back and brought the missing items within an hour. I could not find out how to do that as I have difficulty translating the website into english Hello your chat with Raffaela. Good of you to send a message. Do you have an order number for me? Yes just a minute! 6058655116 this is the order number for the order :) by the way, will I get any points from this order? Okay let's get it out. Have a nice evening"

In [46]:
cleaned = clean_text(text1)

In [48]:
print(extract_text(cleaned))

Hallo staan er voedingswaarden vermeld op de flessen wijn die u verkoopt Hallo je chat met Sharon Bedankt voor je bericht Nee niet volledig Je kan dit navragen in de winkel en anders kunnen wij dit voor je opzoeken Fijne dag Ok bedankt jij ook een fijne dag Groetjes Greet Dankjewel


In [48]:
print(extract_text(text2))

Hello, is it possible to have this conversation in english? Hello! I have made a refund request on Jumbo. I would like to cancel it as the delivery man came back and brought the missing items within an hour. I could not find out how to do that as I have difficulty translating the website into english 


In [49]:
text2

"Hello, is it possible to have this conversation in english? Hello! I have made a refund request on Jumbo. I would like to cancel it as the delivery man came back and brought the missing items within an hour. I could not find out how to do that as I have difficulty translating the website into english Hello your chat with Raffaela. Good of you to send a message. Do you have an order number for me? Yes just a minute! 6058655116 this is the order number for the order :) by the way, will I get any points from this order? Okay let's get it out. Have a nice evening"